In [1]:
import numpy as np

### We start by defining the classes for the vertices and the network and the functions we will need.

In [2]:
class Vertex:
    
    # To initialize a vertex with a given name, initial excess flow and height equal to zero.
    # We also add Adj_h, the set of the names of the vertices v adjacent to self in Gf which have
    # height v.h <= self.h -1. We also prepare the vertex for the structure of the double linked list
    # we will use to store the overflowing vertices of G. We used this as implementation in order
    # to have a better running time.
    def __init__(self,name):
        self.name = str(name)
        self.Adj = {}
        self.e = 0
        self.h = 0
        self.Adj_h = set()
        self.ofnext = None 
        self.ofprev = None
        
    def get_name(self):
        return self.name
        
    def get_Adj(self):
        return self.Adj
    
    def get_h(self):
        return self.h
    
    # To link self to another vertex v through an edge with capacity c.
    # We can also add if the link is directed (True) so the edge has direction from self to v,
    # otherwise, the same edge is also created from v to self
    def link(self, v, c, direct=True):
        self.get_Adj()[v.get_name()] = c
        if not direct:
            v.get_Adj()[self.get_name()] = c
    
    # To get the capacity of the edge from self to v
    def get_c(self,v):
        if v.get_name() not in self.get_Adj():
            return 0
        return self.get_Adj()[v.get_name()]
    
    # The following functions are for the structure of the double linked list
    
    def get_prev(self):
        return self.ofprev
    
    def set_prev(self,prev):
        self.ofprev = prev
        
    def get_next(self):
        return self.ofnext
    
    def set_next(self,next):
        self.ofnext = next

In [3]:
# We now define the class network, where the vertices defined above are linked
# together following the capacity matrix c

class Network:
    
    def __init__(self, Vnames, c, s=None, t=None):
        '''
        Vnames is the list of vertices names: we use them to contruct the vertices with the class defined
        above and we store them in a dictionary where we can access them by their name
        s and t are respectively the source and the sink of the network:
        if not specified, we set them to be respectively the first and the last element in V
        c is the matrix of capacities, its size must be coherent with the length of V
        '''
        self.V = {}
        for name in Vnames:
            self.V[name] = Vertex(name)
        if not s:
            s = self.V[Vnames[0]]
        if not t:
            t = self.V[Vnames[-1]]
        self.s = s
        self.t = t
        self.ofhead = None # We set the head of the linked list of overflowing vertices
        # We also implement a nested dictionary to store the values of the flow,
        # inizialized at 0, and link the vertices
        self.f = {}
        for (i,u) in enumerate(Vnames):
            self.f[u] = {}
            for (j,v) in enumerate(Vnames):
                if c[i][j] >0:
                    self.V[u].link(self.V[v], c[i][j])
                    self.f[u][v] = 0
                    
    # To add a vertex v to the head of the double linked list of overflowing vertices
    def addof(self,v):
        if v != self.t.get_name(): # The sink cannot be overflowing
            if not self.ofhead: # The double linked list was empty: we just inizialize it 
                self.ofhead = v
            else: # We need to link the vertices
                self.V[self.ofhead].set_prev(v)
                self.V[v].set_next(self.ofhead)
                self.ofhead = v
            
    # To remove the head of the double linked list of overflowing vertices
    def popof(self):
        new_head = self.V[self.ofhead].get_next()
        self.V[self.ofhead].set_next(None)
        if not new_head: # There was no linked vertex
            self.ofhead = None
        else:
            self.V[new_head].set_prev(None)
            self.ofhead = new_head
        
    # To get the flow from u to v
    def get_f(self,u,v):
        u = str(u); v = str(v)
        if self.get_c(u,v) == 0:
            return 0
        return self.f[u][v]
    
    # To get the capacity of the edge (u,v)
    def get_c(self,u,v):
        u = str(u); v = str(v)
        return self.V[u].get_c(self.V[v])
        
    # To get the residual capacity of the edge (u,v),
    # that is how much the flow f on this edge can be pushed
    def get_cf(self,u,v):
        u = str(u); v = str(v)
        if self.get_f(u,v) > 0:
            return self.get_c(u,v) - self.get_f(u,v)
        else:
            return self.get_c(u,v) + self.get_f(v,u)
        
    # To get the ajacency set of u
    def get_Adj(self,u):
        return self.V[u].get_Adj().keys()

    # To initialize the preflow: the initial flows, excess flows and heights are 0 by default.
    # We set, for all vertices adjacent to the source s, the flow and excess flow equal to the initial capacity.
    # We also set the height of the source to be equal to the total number of vertices.
    def initialize_preflow(self):
        self.s.h = len(self.V.keys())
        for v in self.s.get_Adj():
            c = self.s.get_c(self.V[v])
            self.f[self.s.get_name()][v] = c
            self.s.e -= c
            self.V[v].e += c
            self.addof(v)
            
    # To add flow from u to v. We recall that increasing the flow
    # on the edge (u,v) is like decreasing it on the edge (v,u)
    def add_f(self,u,v,value):
        u = str(u); v = str(v)
        if self.get_f(v,u) > value:
            self.f[v][u] -= value
        else:
            self.f[u][v] += value - self.get_f(v,u)
            if self.f[u][v] > self.get_c(u,v):
                raise Error('Wrong')
            if self.get_f(v,u) > 0:
                self.f[v][u] = 0
            
    # To push flow from u, the head of the overflowing list,
    # to v, an element of the set u.Adj_h
    def push(self,u,v):
        u = str(u); v = str(v)
        delta_f = min(self.get_cf(u,v), self.V[u].e)
        self.add_f(u,v, delta_f)
        if self.get_cf(u,v) == 0:
            self.V[u].Adj_h.discard(v)
        self.V[u].e -= delta_f
        # if u is no longer overflowing, we remove it from the overflowing list
        if self.V[u].e == 0: 
            self.popof() 
        self.V[v].e += delta_f
        # if v was not overflowing and now it is, we need to add it to the overflowing list
        if self.V[v].e > 0 and not (v == self.ofhead or self.V[v].get_prev()):
            self.addof(v)
            
    # To increase the height of u when u is overflowing and no push operation can be performed,
    # i.e. u.e > 0 and for all (u,v) s.t. c_f(u,v) > 0, u.h <= v.h
    def relabel(self,u):
        u = str(u)
        if self.s.get_name() == u:
            raise Error('Wrong')
        h = float('Inf')
        for v in self.V:
            if self.get_cf(u,v) > 0:
                h = min(h, self.V[v].h)
        self.V[u].h = h + 1
        # we now need to update u.Adj_h 
        self.V[u].Adj_h = set() # clean up the list
        for v in self.V: # update it
            if self.get_cf(u,v) > 0 and self.V[v].h < self.V[u].h:
                self.V[u].Adj_h.add(v)
                self.V[v].Adj_h.discard(u)
    
    # To initialize the prefloow and perform a sequence of push and relabel operations
    # as long as an overflowing vertex exists
    def PushRelabel(self):
        self.initialize_preflow()
        while self.ofhead: # the overflowing linked list is not empty
            currentAdj = self.V[self.ofhead].Adj_h
            if not currentAdj:
                self.relabel(self.ofhead)
            else:
                self.push(self.ofhead,currentAdj.pop())
                    
    # To get the residual graph Gf
    def get_Gf(self):
        cf = np.array([[self.get_cf(u,v) for v in self.V] for u in self.V])
        return Network(self.V.keys(), cf, s=self.s, t=self.t)
    
    # To get the partition S of a minimun cut (S,T) given a graph with maximum flow
    # To know reachability we use a Depth First Search
    def get_S(self):
        Gf = self.get_Gf()
        S = set()
        stack = [Gf.s.get_name()]
        while stack:
            u = stack.pop()
            S.add(u)
            for v in Gf.get_Adj(u):
                if v not in S:
                    stack.append(v)
        return S

### Now we apply the Push-Relabel algorithm to the graph of Zachary's karate club.

In [4]:
# We load the capacity matrix, where each value refers to the number of
# social interactions between to individuals of the club
c = np.loadtxt('ZachData.txt', delimiter=' ')

In [5]:
# We define the vertices names: Mi Hi will be number 1, whereas John A. number 34 
V = [str(u + 1) for u in range(34)]

In [6]:
# We create the network
KarateClub = Network(V, c)

Let us now use Ford-Fulkerson's algorithm to compute the cut $(S,T)$

In [7]:
KarateClub.PushRelabel()

In [8]:
# We print the partition S of the minimum cut (S,T) and we see that we get
# the same result as with Ford-Fulkerson algorithm
S = KarateClub.get_S()
print(S)

{'20', '1', '14', '11', '5', '2', '4', '13', '6', '8', '18', '12', '17', '7', '22', '3'}


In [9]:
# We print T
print(set(V).difference(S))

{'23', '10', '33', '29', '28', '19', '21', '27', '25', '16', '9', '31', '34', '26', '24', '15', '30', '32'}
